# Project: 2DEG mobility plottings (300K)

# 1. Settings

## 1.1 Import modules

In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colors, cm
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
from pathlib import Path
mobilitypy_tutorial_path = Path().absolute()
sys.path.insert(0, os.path.join(mobilitypy_tutorial_path, '../'))
from mobilitypy import Plottings, PlotQuasi3DFuns

In [4]:
save_figure_dir = os.path.join(mobilitypy_tutorial_path,'../../OUTPUTS/FIGS')
savefigure = True
FigFormat = 'png'
FigFormatPaper = 'eps'
fig_save_dpi = 300
color_map_plot = 'viridis'
print_align_space = 20

In [5]:
folder_2deg_data_parent = '/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/NextnanoSimulations/DATAs/AlGaN_HEMT/AlGaN_AlGaN_AlN_HEMT/nnp/'
folder_2deg_density_file = os.path.join(folder_2deg_data_parent, '2DEHG_density/OneDim', 'sim_post_process_data.xlsx')
folder_2deg_mobility_file = os.path.join(folder_2deg_data_parent, '2DEHG_mobility/OneDim', 'sim_post_process_data.xlsx')

## 1.2 Set physical constants

In [6]:
# n_2d => in nm^-2
# rms_roughness => nm^-1
# corr_len => nm^-1
# n_dis => nm^-2
# f_dis => unit less

In [7]:
plot_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT', 'LFOM', 'LFOMnorm', '2DEG']
plots_list_individual = ['TOT', 'LFOM', 'LFOMnorm', '2DEG', '2DHG', 'R']
len_plots = len(plot_mobilities)
rescale_2deg_fact=1e13

x_label_text = 'Al composition barrier, y'
x_label_text_2d = 'Barrier thickness, L$_\\mathrm{B}$ (nm)'
x_p_label_text = 'Al composition channel, x'
y_label_text = 'Barrier thickness, L$_\\mathrm{B}$(nm)'
z_label = {'General': r'2DEG mobility, $\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)', 
           'TOT': r'2DEG mobility, $\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)',
           'LFOM': r'LFOM (MW/cm$^2$)',
           'LFOMnorm': r'LFOM$_\mathrm{(Al,Ga)N}$/LFOM$_\mathrm{highest}$', 
           'LFOMnorm_2': r'LFOM$_\mathrm{(Al,Ga)N}$/LFOM$_\mathrm{GaN}$',
           '2DEG': r'2DEG density, n$_{\mathrm{2D}}$ ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)', 
           '2DHG': r'2DHG density ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)',
           'R': r'Sheet resistance, R$_\mathrm{sh}$ ($\Omega /\boxdot$)'} #Sheet resistance

In [8]:
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize':20,
          'xtick.major.width':2,
          'xtick.major.size':5,
          'xtick.minor.width':2,
          'xtick.minor.size':3,
          'ytick.labelsize': 20,
          'ytick.major.width':2,
          'ytick.major.size':5,
          'ytick.minor.width':2,
          'ytick.minor.size':3,
          'errorbar.capsize':2}
plt.rcParams.update(params)
plt.rc('font', size=20)

# 2. Read mobilities, FOMs from saved database

## 2.1 Al0.25Ga0.75N(25nm)/GaN(305nm) HEMT LFOM for normalization

In [9]:
mobility_ref_dff = pd.read_excel(folder_2deg_mobility_file, index_col=0, sheet_name='ref_Al25Ga75N_GaN') 

In [10]:
mobility_ref_df = mobility_ref_dff[mobility_ref_dff['T'] == 300].copy() # Reference data at 300 K
ref_LFOM = mobility_ref_df['LFOM'].iloc[0]

Conclusion: 2DEG density negligible depend on temperature

In [11]:
print(f'{"- Reference sample":<{print_align_space}}: Al0.25Ga0.75N(25nm)/GaN(305nm)')
print(f'{"-- 2DEG density":<{print_align_space}}: {mobility_ref_df['2DEG_device'].iloc[0]:.5e} cm^-2')
print(f'{"-- 2DEG mobility":<{print_align_space}}: {mobility_ref_df['TOT'].iloc[0]:.5e} cm^2.V^-1.s^-1')
print(f'{"-- LFOM":<{print_align_space}}: {ref_LFOM:.5e} MW/cm^2')

- Reference sample  : Al0.25Ga0.75N(25nm)/GaN(305nm)
-- 2DEG density     : 6.25000e+12 cm^-2
-- 2DEG mobility    : 1.65224e+03 cm^2.V^-1.s^-1
-- LFOM             : 2.63809e+04 MW/cm^2


## 2.2 AlGaN/AlGaN mobilities and figure-of-merit

In [12]:
mobility_dff = pd.read_excel(folder_2deg_mobility_file, index_col=0, sheet_name='AlGaN_AlGaN') #, dtype=float) 
## Not sure why 'IFR' column is read as object
mobility_dff['IFR'] = pd.to_numeric(mobility_dff['IFR'], errors='coerce')
# Collect data for 300 K temperature only
mobility_dff_300k = mobility_dff[mobility_dff['T'] == 300].copy()

In [13]:
#mobility_dff.dtypes

In [14]:
mobility_dff_300k['2DEG'] =  mobility_dff_300k['2DEG_device']/rescale_2deg_fact
mobility_dff_300k['2DHG'] =  mobility_dff_300k['2DHG_device']/rescale_2deg_fact
mobility_dff_300k['LFOMnorm'] = mobility_dff_300k['LFOM']/ref_LFOM

In [15]:
# mobility_df_300k = mobility_dff_300k.copy()

# # Put 10^10 for large values and np.nan for values < 1
# clean_up_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT']
# for which_column in clean_up_mobilities:
#     mobility_df_300k.loc[mobility_df_300k[which_column] <10, which_column] = np.nan 

# # Put 10^10 for large values and np.nan for values < 1
# for which_column in ['2DEG', 'LFOM', 'LFOMnorm', 'LFOM_2DEG1e13']:
#     mobility_df_300k.loc[mobility_df_300k[which_column] <1e-3, which_column] = np.nan 

# # Put 10^10 large values to np.nan for R
# for which_column in ['R']:
#     mobility_df_300k.loc[mobility_df_300k[which_column] >1e5, which_column] = np.nan

In [16]:
mobility_df_300k = mobility_dff_300k.copy()

clean_up_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT']
XXX_ = mobility_df_300k['2DEG'] <1e-2 # Put np.nan for 2DEG densities < 1e-2
for which_column in ['2DEG', 'LFOM', 'LFOMnorm', 'LFOM_2DEG1e13']+clean_up_mobilities+['R']:
    mobility_df_300k.loc[XXX_, which_column] = np.nan 

## 2.4 Plot AlGaN/AlGaN HEMT mobilities

In [17]:
tick_multiplicator = [0.1, 0.05, 0.1, 0.05]

In [18]:
df_cap_thickness_group = mobility_df_300k.groupby(['ThicknessAlGaNBarrier']) # Group by thickness

### 2.4.1 All contributions together

In [58]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities/Together')

In [42]:
fig_ncols, fig_nrows = 3, 3
for name, group in df_cap_thickness_group:
    barrier_thickness = f'{name[0]:.2f}'
    print(f'- Plotting barrier thickness = {barrier_thickness} nm')
    fig, axs = plt.subplots(fig_ncols, fig_nrows, figsize=(16.,12.5), 
                            sharey=True,constrained_layout=True)    
    for ii in range(fig_ncols*fig_nrows):
        if ii < len(plot_mobilities):
            which_mobility = plot_mobilities[ii]
        else:
            #axs[ii//3][ii%3].axis("off") 
            continue
        print(f'\t-- Plotting {which_mobility}...')
        xx, yy, zz = group['AlContentChannel'], group['AlContentBarrier'], group[which_mobility]
        vmax = min(mobility_df_300k[which_mobility].max(numeric_only=True), 1e10)
        vmin = mobility_df_300k[which_mobility].min(numeric_only=True) if which_mobility in ['2DEG', 'LFOM', 'LFOMnorm'] else 20
        #vmin, vmax = zz.min()+1e-20, zz.max()
        norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['LFOMnorm','2DEG'] else colors.LogNorm(vmin=vmin, vmax=vmax)
        cbar_mapable = cm.ScalarMappable(norm=norm, cmap='viridis')
        x_label_text_ = x_p_label_text #if ii-6>=0 else ''
        y_label_text_ = x_label_text #if ii%3==0 else ''
        z_label_ = ''
        #if ii%3-2==0: 
        z_label_ = f'$\\mu_{{\\mathrm{{{which_mobility}}}}}$ ($\\mathrm{{cm}}^2\\mathrm{{V}}^{{-1}}\\mathrm{{s}}^{{-1}}$)'
        if which_mobility in ['2DEG']: z_label_= r'n$_{\mathrm{2D}}$ ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)'
        if which_mobility in ['LFOM']: z_label_= z_label[which_mobility]
        if which_mobility == 'LFOMnorm': z_label_= z_label[f'{which_mobility}_2'] 
        

        ##### Plot composition map 
        fig, axt = lpltq3d.Plotq3D(xx,yy,zz, fig=fig, ax=axs[ii//3][ii%3],
                                 xmin=0.475, xmax=0.975, ymin=0.525, ymax=1.025,
                                 x_label='', y_label=y_label_text_,
                                 interpolation_method='linear',
                                 interpolation_points = 20,
                                 tick_multiplicator=tick_multiplicator,
                                 title_label=None, #which_mobility,
                                 cbar_mappable=cbar_mapable, norm=norm,
                                 show_contour_lines=False, marker='s', marker_size=24**2,
                                 cbar_text=z_label_,show_colorbar=False,
                                 plot_controur=0, plot_scatter=1,
                                 savefigure=False, show_plot=False)
        #plt.suptitle(f"Barrier thickness = {barrier_thickness} nm", size=24)
        axt.set_xlabel(x_label_text_, size=18)
        axt.set_ylabel(y_label_text_, size=18)
        cbar = fig.colorbar(cbar_mapable, ax=axt)         
        cbar.ax.set_ylabel(z_label_, size=18)
        #if ii%3 != 0: axs[ii//3][ii%3].get_yaxis().set_visible(False)
    lpltq3d.save_figure(f'Barrier_{barrier_thickness}.{FigFormat}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)
    print('- Done', flush=True)
    #break

- Plotting barrier thickness = 5.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 10.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 15.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 20.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 25.00 nm
	-- Plotting IFR...
	-- Plotting 

### 2.4.2 Individual plots

In [59]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities/Individual')

In [44]:
for name, group in df_cap_thickness_group:
    #if name[0] < 49: continue
    barrier_thickness = f'{name[0]:.2f}'
    print(f'- Plotting barrier thickness = {barrier_thickness} nm')
    for which_mobility in plots_list_individual:
        print(f'\t-- Plotting {which_mobility}...')
        xx, yy, zz = group['AlContentChannel'], group['AlContentBarrier'], group[which_mobility]
        vmax = min(mobility_df_300k[which_mobility].max(numeric_only=True), 1e10)
        vmin = mobility_df_300k[which_mobility].min(numeric_only=True) #if which_mobility in ['2DEG', 'LFOMnorm', 'R'] else 20 
        norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['2DEG', '2DHG', 'LFOMnorm', 'TOT'] else colors.LogNorm(vmin=vmin, vmax=vmax)
        cbar_mapable = cm.ScalarMappable(norm=norm, cmap=color_map_plot)
        z_label_= z_label[which_mobility]
        if which_mobility == 'LFOMnorm': z_label_= z_label[f'{which_mobility}_2'] 
        FigFormat_tmp = FigFormatPaper if which_mobility in ['2DEG','2DHG'] else FigFormat
        ##### Plot composition map 
        _ = lpltq3d.Plotq3D(xx, yy, zz, x_label=x_p_label_text, y_label=x_label_text,
                            xmin=0.475, xmax=0.975, ymin=0.525, ymax=1.025,
                            interpolation_method='linear', interpolation_points = 20,
                            tick_multiplicator=tick_multiplicator, norm=norm,color_map=color_map_plot,
                            cbar_mappable=cbar_mapable, show_contour_lines=1,
                            marker='s', marker_size=38**2, cbar_text=z_label_,
                            show_colorbar=True, plot_controur=0, plot_scatter=1,
                            save_file_name=f'Barrier_{barrier_thickness}_{which_mobility}.{FigFormat_tmp}',
                            savefigure=savefigure, show_plot=False, dpi=fig_save_dpi)
    print('- Done', flush=True)
    #break

- Plotting barrier thickness = 5.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting 2DHG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 10.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting 2DHG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 15.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting 2DHG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 20.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting 2DHG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 25.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting 2DHG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 30.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plott

In [60]:
#### This is specific for plottings for paper
for name, group in df_cap_thickness_group:
    barrier_thickness = f'{name[0]:.2f}'
    if name[0] < 49: continue
    for which_mobility in ['2DEG', 'TOT', 'LFOMnorm', 'LFOM', 'R']: #
        xx, yy, zz = group['AlContentChannel'], group['AlContentBarrier'], group[which_mobility]
        vmax = min(zz.max(numeric_only=True), 1e10)
        vmin = zz.min(numeric_only=True) #if which_mobility in ['2DEG', 'LFOMnorm', 'R'] else 20 
        norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['2DEG', 'LFOMnorm', 'LFOM', 'TOT', 'R'] else colors.LogNorm(vmin=vmin, vmax=vmax)
        cbar_mapable = cm.ScalarMappable(norm=norm, cmap=color_map_plot)
        z_label_= z_label[which_mobility]
        if which_mobility == 'LFOMnorm': z_label_= z_label[f'{which_mobility}_2'] 
        show_colorbar_ = True
        savefigure_ = savefigure
        if which_mobility in ['LFOM','R']:
            show_colorbar_=False
            savefigure_ = False
        figsize=(7.9,6)
        if which_mobility in ['TOT']: figsize=(8.1,6)
        fig, ax = plt.subplots(figsize=figsize, constrained_layout=True)
        ##### Plot composition map 
        fig, ax = lpltq3d.Plotq3D(xx, yy, zz, fig=fig, ax=ax, x_label=x_p_label_text, y_label=x_label_text,
                            xmin=0.475, xmax=0.975, ymin=0.525, ymax=1.025,
                            interpolation_method='linear', interpolation_points = 100,
                            tick_multiplicator=tick_multiplicator, norm=norm,color_map=color_map_plot,
                            cbar_mappable=cbar_mapable, show_contour_lines=1,
                            marker='s', marker_size=38**2, cbar_text=z_label_,
                            show_colorbar=show_colorbar_, plot_controur=0, plot_scatter=1,
                            save_file_name=f'Barrier_{barrier_thickness}_{which_mobility}_Paper.{FigFormatPaper}',
                            savefigure=savefigure_, show_plot=False, dpi=fig_save_dpi)
        if which_mobility in ['LFOM', 'R']:
            cbar = fig.colorbar(cbar_mapable, ax=ax)         
            cbar.ax.set_ylabel(z_label_)
            cbar.formatter.set_powerlimits((0, 0))
            # to get 10^3 instead of 1e3
            cbar.formatter.set_useMathText(True)
            cbar.ax.yaxis.set_offset_position('left')
            cbar.update_ticks()

        lpltq3d.save_figure(f'Barrier_{barrier_thickness}_{which_mobility}_Paper.{FigFormatPaper}', 
                            savefig=not savefigure_, show_plot=False, fig=fig, CountFig=None, dpi=fig_save_dpi) 
        
    print('- Done', flush=True)
    #break

- Done


In [61]:
plt.close('all')

In [19]:
plt2deg = Plottings(save_figure_dir=f'{save_figure_dir}/Others')

In [48]:
output_dataframe = {}
fig, ax = plt.subplots(figsize=(8,7))
tick_multiplicator = [0.1, 0.05, 1, 0.5]
markers_ = ['o', '*', 'd', '^', 's', 'p']*3
ii=0
for name, group in mobility_dff_300k.groupby(['ThicknessAlGaNBarrier']):
    output_dataframe[f'{name[0]:.2f}'] = {}
    group['AlContentContrast'] = (group['AlContentBarrier'] - group['AlContentChannel']) #/ group['AlContentChannel']
    
    #=====================================================================================
    if name[0] in [5,10,20,50]:
        ax.scatter(group['AlContentContrast'], group['2DEG'], marker=markers_[ii],label=f'{name[0]:.2f} nm',s=100)
        ii+=1
    
    #=====================================================================================
    tmp_df = group[group[f'2DEG']>0.01]
    A = np.vstack([tmp_df['AlContentContrast'], np.ones(len(tmp_df))]).T
    m, c = np.linalg.lstsq(A, tmp_df[f'2DEG'])[0]
    y_eq_zero_point = -c/m
    output_dataframe[f'{name[0]:.2f}'][f'2DEG_slope']       = f'{m:.3f}' 
    output_dataframe[f'{name[0]:.2f}'][f'2DEG_y_intersect'] = f'{c:.3f}'
    output_dataframe[f'{name[0]:.2f}'][f'2DEG_x_intersect'] = f'{y_eq_zero_point:.3f}'
    new_x = np.insert(np.array(tmp_df['AlContentContrast']),0,y_eq_zero_point)
    #ax.plot(new_x, m*new_x + c, 'gray')
    if name[0] in [5,10,20,50]: 
        ax.plot(new_x, m*new_x + c, 'gray')
    
    ax.legend(ncols=1, labelspacing=0.001, columnspacing=0.1, handletextpad=0.01)
    ax.set_xlabel('Al composition contrast, $\\Delta_{yx}$')
    ax.set_ylabel(z_label['2DEG'])
    ax.axhline(color='k',ls='--')
ax.axhline(y=1, c='c', ls='--')
ax.set_xlim(0.05,0.5)
plt2deg.set_tickers(ax, tick_multiplicator)
circle = Ellipse((0.35,1.6), width=0.025, height=0.35, 
                        edgecolor='r', fc='None', lw=2)
ax.add_patch(circle)
plt2deg.save_figure(f'Comp_contrast_thickness_2DEG.{FigFormatPaper}', savefig=savefigure, show_plot=True,
                    fig=fig, CountFig=None, dpi=fig_save_dpi)      

/tmp/ipykernel_2643920/2288647217.py:18: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  m, c = np.linalg.lstsq(A, tmp_df[f'2DEG'])[0]
/tmp/ipykernel_2643920/2288647217.py:18: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  m, c = np.linalg.lstsq(A, tmp_df[f'2DEG'])[0]
/tmp/ipykernel_2643920/2288647217.py:18: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass 

In [49]:
#=============================================================================================
output_data = pd.DataFrame.from_dict(output_dataframe, orient='index', dtype=float)
## Plot cut-off composition contrast for each barrier thickness
tick_multiplicator = [0.1, 0.05,10,5]
XX = np.array(output_data.index, dtype=float)
YY = np.array(output_data['2DEG_x_intersect'], dtype=float)
fig, axs = plt.subplots(constrained_layout=True)
axs.plot(YY, XX, 'ko-', ms=12)
#axs.set_yscale('log')
#axs.set_ylabel('Critical barrier thickness (nm)')
#axs.set_xlabel('Critical composition contrast')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left', title='Al-content') 
axs.set_xlim(0.05, 0.54)
plt2deg.set_tickers(axs, tick_multiplicator)
 

fit_intercept = np.array(output_data['2DEG_y_intersect'], dtype=float)
fit_slope = np.array(output_data['2DEG_slope'], dtype=float)
YY = (1 - fit_intercept)/fit_slope
axs.plot(YY, XX, 'cs-', ms=12)
#axs.set_yscale('log')
axs.set_ylabel('Critical barrier thickness (nm)')
#axs.set_ylabel('Barrier thickness, L$_\\mathrm{B}$ (nm)')
axs.set_xlabel('Al composition contrast, $\\Delta_{yx}$')

plt2deg.save_figure(f'Critical_comp_contrast_2DEG.{FigFormatPaper}', savefig=savefigure, show_plot=True,
                    fig=fig, CountFig=None, dpi=fig_save_dpi) 

# _ = lplt1d.PlotSweepsData(XX, YY, x_label=, y_label=,
#                           tick_multiplicator=tick_multiplicator,
#                           FigDpi=FigDpi, FigFormat=FigFormat, color='k', marker='o',
#                           figs_path=f'{output_figs_sweep}/Others', filename=f'Critical_comp_contrast_2DEG',
#                           savefigure=savefigure)

## 2.5 Plots for AlN/AlGaN

In [20]:
plt2deg = Plottings(save_figure_dir=f'{save_figure_dir}/Others')

### 2.5.1 Mobility plot

In [21]:
mobility_df_300k_ = mobility_dff_300k[mobility_dff_300k['AlContentBarrier']>0.99]
df_channel_comp_group = mobility_df_300k_.groupby(['AlContentChannel']) # Group by channel composition

In [22]:
x_label = 'Barrier thickness, L$_\\mathrm{B}$ (nm)'
y_label = z_label['TOT'] #r'Electron mobility ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'
save_file_name_ = f'mu_Tb_AlN_AlGaN.{FigFormatPaper}'
tick_multiplicator=[10, 5, 50, 25]
plot_selected_compositions = ['0.50', '0.65', '0.75', '0.85', '0.90', '0.95']
markers_ = ['o', '*', 's', '^', 'd', 'p']*3

In [67]:
fig, axs = plt.subplots(constrained_layout=True)
ii = 0
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['TOT']
    if f'{name[0]:0.2f}' in plot_selected_compositions: 
        axs.plot(xx, zz, '-', marker=markers_[ii],label=f'x={name[0]:.2f}', ms=12)
        ii+=1
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left', title='Al-content') 
axs.set_xlim(5,50)
plt2deg.set_tickers(axs, tick_multiplicator)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [68]:
save_file_name_ = f'channel_comp_legends.{FigFormatPaper}'
# Now create a new image with legends only
# adjust the figure size as necessary
fig_leg = plt.figure(figsize=(1,1))
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*axs.get_legend_handles_labels(), ncol=3, loc='center')
# hide the axes frame and the x/y labels
ax_leg.axis('off')
#ax_leg.set_frame_on(False)
plt2deg.save_figure(save_file_name_, fig=fig_leg, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [69]:
fig, axs = plt.subplots(constrained_layout=True) #figsize=(9.5,6)
ii = 0
for name, group in df_channel_comp_group:
    if name[0]< 0.74 or name[0]> 0.91: 
        axs.plot([None], [None], label=None)
        continue
    xx, zz = group['ThicknessAlGaNBarrier'],  group['TOT']
    axs.plot(xx, zz, '-', marker=markers_[ii],label=f'{name[0]:.2f}', ms=12)
    ii+=1
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(f'zoom_{save_file_name_}', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.2 Individual mobility contributions for AlN/Al0.9Ga0.1N

In [70]:
Al90Ga10N_mu_df_300k_ = mobility_df_300k_[(mobility_df_300k_['AlContentChannel']>0.89) & (mobility_df_300k_['AlContentChannel']<0.91)]

In [72]:
fig, axs = plt.subplots(figsize=(9.5,6),constrained_layout=True)
ii=0
for name in plot_mobilities[:-3]:
    axs.plot(Al90Ga10N_mu_df_300k_['ThicknessAlGaNBarrier'], Al90Ga10N_mu_df_300k_[name], '-', marker=markers_[ii],label=f'{name}', ms=12)
    ii+=1
axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
axs.set_ylim(100, 100000)
axs.set_xlim(5,50)
plt2deg.set_tickers(axs, [10,5,None, None])
plt2deg.save_figure(f'mu_contibs_Tb_AlN_Al0.9Ga0.1N.{FigFormat}', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.3 FOM plot

In [23]:
x_label = x_label_text_2d
y_label = z_label['LFOMnorm_2']
save_file_name_ = f'lfom_Tb_AlN_AlGaN.{FigFormatPaper}'
tick_multiplicator=[10, 5, 0.5, 0.25]

In [75]:
fig, axs = plt.subplots(constrained_layout=True)
ii=0
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['LFOMnorm']
    if f'{name[0]:0.2f}' in plot_selected_compositions:
        axs.plot(xx, zz, '-', marker=markers_[ii],label=f'{name[0]:.2f}', ms=12)
        ii+=1
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [76]:
fig, axs = plt.subplots(constrained_layout=True)
ii=0
for name, group in df_channel_comp_group:
    if name[0]< 0.74 or name[0]> 0.91: 
        axs.plot([None], [None])
        continue
    xx, zz = group['ThicknessAlGaNBarrier'],  group['LFOMnorm']
    axs.plot(xx, zz, '-', marker=markers_[ii],label=f'{name[0]:.2f}', ms=12)
    ii+=1
#axs.set_yscale('log')
axs.axhline(y=1.0, c='k', ls='--')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(f'zoom_{save_file_name_}', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.4 2DEG density plot

In [77]:
x_label = x_label_text_2d
y_label = z_label['2DEG']
save_file_name_ = f'deg_Tb_AlN_AlGaN.{FigFormatPaper}'
tick_multiplicator=[10, 5, None, None]

In [78]:
fig, axs = plt.subplots(figsize=(8,6),constrained_layout=True)
ii=0
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['2DEG']
    if f'{name[0]:0.2f}' in plot_selected_compositions:
        axs.plot(xx, zz, '-', marker=markers_[ii],label=f'{name[0]:.2f}', ms=12)
        ii+=1
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [80]:
fig, axs = plt.subplots(figsize=(8,6),constrained_layout=True)
ii=0
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['2DEG']
    if f'{name[0]:0.2f}' in ['0.50']: 
        axs.plot(xx, zz, 'r-', marker=markers_[ii],label=f'{name[0]:.2f}', ms=12)
        ii+=1
axs.set_xlabel(x_label_text_2d)
axs.set_ylabel(y_label)
axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, [10, 5, None, None])
#axs.set_xlim(5,50)
plt2deg.save_figure(f'two_deg_Tb_AlN_Al0.50GaN.{FigFormatPaper}', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.4 Sheet resitance plot

In [82]:
x_label = x_label_text_2d
y_label = z_label['R']
save_file_name_ = f'R_Tb_AlN_AlGaN.{FigFormat}'
tick_multiplicator=[10, 5, None, None]

In [44]:
fig, axs = plt.subplots(figsize=(8,6),constrained_layout=True)
ii=0
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['R'].copy()
    zz[zz>1e8] = np.nan
    if f'{name[0]:0.2f}' in plot_selected_compositions:
        axs.plot(xx, zz, '-', marker=markers_[ii],label=f'{name[0]:.2f}', ms=12)
        ii+=1
axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [45]:
fig, axs = plt.subplots(figsize=(8,6),constrained_layout=True)
ii=0
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['R'].copy()
    zz[zz>1e8] = np.nan
    if f'{name[0]:0.2f}' in plot_selected_compositions:
        axs.plot(xx, zz, '-', marker=markers_[ii],label=f'{name[0]:.2f}', ms=12)
        ii+=1
axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
axs.set_ylim(ymax=5e4)
plt2deg.save_figure(f'zoom_{save_file_name_}', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.5 Mobility contributions for AlN(50nm)/AlGaN

In [24]:
aln_algan_300k = mobility_dff_300k[(mobility_dff_300k['AlContentBarrier']>0.99) & (mobility_dff_300k['ThicknessAlGaNBarrier']>49)]
aln_algan_dff = aln_algan_300k[['AlContentChannel','IFR','AD','DIS','DP','PE','AP','POP','TOT']].copy()
aln_algan_dff.rename(columns={'AlContentChannel':'comp'}, inplace=True)

x_label = x_p_label_text
y_label = z_label['TOT']
save_file_name_ = f'mu_AlN50_AlGaN.{FigFormatPaper}'
tick_multiplicator=[0.1, 0.05, None, None]

In [84]:
# Mobility contributions at 300K
save_file_name_ = f'Mobility_contribs_AlN50AlGaN_300K.{FigFormatPaper}'
x_label = x_p_label_text
y_label = z_label['TOT']
fig, ax,_ = plt2deg.plot_2d_carrier_mobilities(aln_algan_dff, save_file_name=save_file_name_,
                                               ymin=5e1, ymax=2e5, xmax=0.9, xmin=0.5, y_scale_log=True, 
                                               annotate_pos=(2,2), annotatetextoffset=(0,-20),show_right_ticks=True,
                                               mode='2d_carrier_mobility', yaxis_label=y_label, xaxis_label=x_label,
                                               color=None, color_map='viridis', savefig=0, show_plot=False)
plt2deg.set_tickers(ax, tick_multiplicator)
plt2deg.save_figure(save_file_name_, fig=fig, savefig= savefigure, dpi=fig_save_dpi, show_plot=False)

In [86]:
fig, ax = plt.subplots(figsize=(9,6), constrained_layout=True)
XX = np.array(aln_algan_300k['AlContentChannel'], dtype=float)
ax.plot(XX, aln_algan_300k['2DEG_device']/rescale_2deg_fact, 'o-', c='k')
ax.set_xlim(0.5,0.9)
ax.set_xlabel(x_p_label_text)
ax.set_ylabel(z_label['2DEG'])
plt2deg.set_tickers(ax, tick_multiplicator)

# Plotting mobility
ax2 = ax.twinx() 
ax2.plot(XX, aln_algan_300k['TOT'], 'o-', c='r')

# Plotting bandgap
bandgap_ = (6.20*XX + 3.43*(1-XX) + 0.7*XX*(1-XX))**5 
ax2.plot(XX, bandgap_, 'o-', c='m')

ax2.set_ylabel(z_label['TOT'], c='r')
ax2.set_yscale('log')
ax2.set_ylim(ymin=70, ymax=1e4)
ax2.annotate(r'E$_\mathrm{g}^{5}$', xy=(XX[3], bandgap_[3]+1e3), color='m')

# Plotting sheet resistance
R_300k = np.array(aln_algan_300k['R'], dtype=float) #/20000 #1/20000 is to rescale
ax2.plot(XX, R_300k, 'o-', c='g')
ax2.annotate(r'R', xy=(XX[3], R_300k[3]-1e3), color='g')

# Plotting LFOM
LFOM_300k = np.array(aln_algan_300k['LFOM'], dtype=float)/ref_LFOM 
ax.plot(XX, LFOM_300k, 'o-', c='b')
ax.annotate(r'LFOM$_{\mathrm{norm}}$', xy=(XX[3], LFOM_300k[3]-0.2), color='b')

plt2deg.save_figure(f'AlN50AlGaN_300K_mu_FOM.{FigFormat}', savefig=savefigure, show_plot=True,
                    fig=fig, CountFig=None, dpi=fig_save_dpi)

In [50]:
fig, ax = plt.subplots(figsize=(8,6), constrained_layout=True)
XX = np.array(aln_algan_300k['AlContentChannel'], dtype=float)
yy_2deg_density = np.array(aln_algan_300k['2DEG_device']/rescale_2deg_fact, dtype=float)
ax.plot(XX, yy_2deg_density, 's-.', c='g', ms=12)
ax.axhline(y=1, c='k', ls='--')
ax.annotate(z_label['2DEG'][14:], xy=(XX[3], yy_2deg_density[3]+0.08), color='g')
circle0 = Ellipse((XX[2], yy_2deg_density[2]), width=0.015, height=0.2, 
                        edgecolor='g', fc='None', lw=2)
ax.add_patch(circle0)
ax.annotate("", xy=(XX[2]-0.05, yy_2deg_density[2]-0.1), xytext=(XX[2], yy_2deg_density[2]-0.1),
             arrowprops=dict(arrowstyle="->",color='g', linewidth=2))
ax.set_xlim(0.5,0.9)
ax.set_xlabel(x_p_label_text)
ax.set_ylabel('')
plt2deg.set_tickers(ax, tick_multiplicator)

#=========================================================================
ax2 = ax.twinx() 
ax2.set_ylabel('', c='r')
ax2.set_yscale('log')
ax2.set_ylim(ymin=50, ymax=1e4)

# Plotting mobility
yy_mobility = np.array(aln_algan_300k['TOT'], dtype=float)
ax2.plot(XX, yy_mobility, '*-.', c='c', ms=12)
ax2.annotate(z_label['TOT'][15:], xy=(XX[3], yy_mobility[3]-25), color='c')
circle1 = Ellipse((XX[6], yy_mobility[6]), width=0.014, height=30, 
                        edgecolor='c', fc='None', lw=2)
ax2.add_patch(circle1)
ax2.annotate("", xy=(XX[6]+0.05, yy_mobility[6]-15), xytext=(XX[6], yy_mobility[6]-15),
             arrowprops=dict(arrowstyle="->",color='c', linewidth=2))

# Plotting bandgap
bandgap_ = (6.25*XX + 3.51*(1-XX) + 0.7*XX*(1-XX))**5 
ax2.plot(XX, bandgap_, '^-.', c='m', ms=12)
ax2.annotate(r'E$_\mathrm{g}^{5}$ (eV$^{5}$)', xy=(XX[3], bandgap_[3]+1.2e3), color='m')
circle2 = Ellipse((XX[6], bandgap_[6]), width=0.015, height=1.6e3, 
                        edgecolor='m', fc='None', lw=2)
ax2.add_patch(circle2)
ax2.annotate("", xy=(XX[6]+0.05, bandgap_[3]+1.e3), xytext=(XX[6], bandgap_[3]+1.e3),
             arrowprops=dict(arrowstyle="->",color='m', linewidth=2))

# Plotting LFOM
LFOM_300k = np.array(aln_algan_300k['LFOM'], dtype=float)/ref_LFOM 
ax.plot(XX, LFOM_300k, 'o-', c='r', ms=12)
ax.annotate(r'LFOM$^{\mathrm{B}}_{\mathrm{norm}}$', xy=(XX[1], LFOM_300k[1]+0.3), color='r')
circle3 = Ellipse((XX[2], LFOM_300k[2]), width=0.015, height=0.2, 
                        edgecolor='r', fc='None', lw=2)
ax.add_patch(circle3)
ax.annotate("", xy=(XX[2]-0.05, LFOM_300k[2]+0.1), xytext=(XX[2], LFOM_300k[2]+0.1),
             arrowprops=dict(arrowstyle="->",color='r', linewidth=2))

LFOM_300k_cnst = np.array(aln_algan_300k['LFOM_2DEG1e13'], dtype=float)/ref_LFOM 
ax.plot(XX, LFOM_300k_cnst, 'd-', c='b', ms=12)
ax.annotate(r'LFOM$^{\mathrm{A}}_{\mathrm{norm}}$', xy=(XX[1], LFOM_300k_cnst[1]+0.3), color='b')
circle4 = Ellipse((XX[2], LFOM_300k_cnst[2]), width=0.015, height=0.2, 
                        edgecolor='b', fc='None', lw=2)
ax.add_patch(circle4)
ax.annotate("", xy=(XX[2]-0.05, LFOM_300k_cnst[2]+0.1), xytext=(XX[2], LFOM_300k_cnst[2]+0.1),
             arrowprops=dict(arrowstyle="->",color='b', linewidth=2))

plt2deg.set_tickers(ax, [0.1,0.05,0.5,0.25])

plt2deg.save_figure(f'AlN50AlGaN_300K_mu_FOM_paper.{FigFormatPaper}', savefig=savefigure, show_plot=True,
                    fig=fig, CountFig=None, dpi=fig_save_dpi)

In [89]:
ii=0
for ii in range(5):
    fig, ax = plt.subplots(figsize=(8,6.), constrained_layout=True)
    XX = np.array(aln_algan_300k['AlContentChannel'], dtype=float)

    if ii >2:
        yy_2deg_density = np.array(aln_algan_300k['2DEG_device']/rescale_2deg_fact, dtype=float)
        ax.plot(XX, yy_2deg_density, 's-', c='k', ms=12)
        ax.annotate(z_label['2DEG'][14:], xy=(XX[3], yy_2deg_density[3]+0.08), color='k')
        circle0 = Ellipse((XX[2], yy_2deg_density[2]), width=0.015, height=0.2, 
                                edgecolor='k', fc='None', lw=2)
        ax.add_patch(circle0)
        ax.annotate("", xy=(XX[2]-0.05, yy_2deg_density[2]-0.1), xytext=(XX[2], yy_2deg_density[2]-0.1),
                     arrowprops=dict(arrowstyle="->",color='k', linewidth=2))
        
    ax.axhline(y=1, c='k', ls='--')
    ax.set_xlim(0.5,0.9)
    ax.set_ylim(-0.12,3.55)
    ax.set_xlabel(x_p_label_text)
    ax.set_ylabel('')
    plt2deg.set_tickers(ax, [0.1,0.05,0.5,0.25])
    
    #=========================================================================
    ax2 = ax.twinx() 
    ax2.set_ylabel('', c='r')
    ax2.set_yscale('log')
    ax2.set_ylim(ymin=50, ymax=1e4)
    
    # Plotting mobility
    if ii >1:
        yy_mobility = np.array(aln_algan_300k['TOT'], dtype=float)
        ax2.plot(XX, yy_mobility, '*-', c='c', ms=12)
        ax2.annotate(z_label['TOT'][15:], xy=(XX[3], yy_mobility[3]-25), color='c')
        circle1 = Ellipse((XX[6], yy_mobility[6]), width=0.014, height=30, 
                                edgecolor='c', fc='None', lw=2)
        ax2.add_patch(circle1)
        ax2.annotate("", xy=(XX[6]+0.05, yy_mobility[6]-15), xytext=(XX[6], yy_mobility[6]-15),
                     arrowprops=dict(arrowstyle="->",color='c', linewidth=2))
    if ii >0:
        # Plotting bandgap
        bandgap_ = (6.25*XX + 3.51*(1-XX) + 0.7*XX*(1-XX))**5 
        ax2.plot(XX, bandgap_, '^-', c='m', ms=12)
        ax2.annotate(r'E$_\mathrm{g}^{5}$ (eV$^{5}$)', xy=(XX[3], bandgap_[3]+1.2e3), color='m')
        circle2 = Ellipse((XX[6], bandgap_[6]), width=0.015, height=1.6e3, 
                                edgecolor='m', fc='None', lw=2)
        ax2.add_patch(circle2)
        ax2.annotate("", xy=(XX[6]+0.05, bandgap_[3]+1.e3), xytext=(XX[6], bandgap_[3]+1.e3),
                     arrowprops=dict(arrowstyle="->",color='m', linewidth=2))
    
    # Plotting LFOM
    LFOM_300k = np.array(aln_algan_300k['LFOM'], dtype=float)/ref_LFOM 
    ax.plot(XX, LFOM_300k, 'o-', c='r', ms=12)
    ax.annotate(r'LFOM$_{\mathrm{norm}}$', xy=(XX[1], LFOM_300k[1]+0.3), color='r')
    circle3 = Ellipse((XX[2], LFOM_300k[2]), width=0.015, height=0.2, 
                            edgecolor='r', fc='None', lw=2)
    ax.add_patch(circle3)
    ax.annotate("", xy=(XX[2]-0.05, LFOM_300k[2]+0.1), xytext=(XX[2], LFOM_300k[2]+0.1),
                 arrowprops=dict(arrowstyle="->",color='r', linewidth=2))
    if ii >3:
        LFOM_300k_cnst = np.array(aln_algan_300k['LFOM_2DEG1e13'], dtype=float)/ref_LFOM 
        ax.plot(XX, LFOM_300k_cnst, 'd-', c='b', ms=12)
        ax.annotate(r'LFOM$^*_{\mathrm{norm}}$', xy=(XX[1], LFOM_300k_cnst[1]+0.3), color='b')
        circle4 = Ellipse((XX[2], LFOM_300k_cnst[2]), width=0.015, height=0.2, 
                                edgecolor='b', fc='None', lw=2)
        ax.add_patch(circle4)
        ax.annotate("", xy=(XX[2]-0.05, LFOM_300k_cnst[2]+0.1), xytext=(XX[2], LFOM_300k_cnst[2]+0.1),
                     arrowprops=dict(arrowstyle="->",color='b', linewidth=2))
    
    plt2deg.save_figure(f'{ii}_AlN50AlGaN_300K_mu_FOM_paper.{FigFormat}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)
    ii+=1
    #break

In [91]:
fig, ax = plt.subplots(constrained_layout=True)
XX = np.array(aln_algan_300k['AlContentChannel'], dtype=float)
#ax.plot(XX, aln_algan_300k['2DEG_device']/rescale_2deg_fact, 'o-', c='k')
ax.set_xlim(0.5,0.9)
ax.set_xlabel(x_p_label_text)
ax.set_ylabel(z_label['2DEG'])
plt2deg.set_tickers(ax, tick_multiplicator)

# # Plotting mobility
# ax2 = ax.twinx() 
# ax2.plot(XX, aln_algan_300k['TOT'], 'o-', c='r')

# # Plotting bandgap
# bandgap_ = (6.20*XX + 3.43*(1-XX) + 0.7*XX*(1-XX))**5 
# ax2.plot(XX, bandgap_, 'o-', c='m')

# ax2.set_ylabel(z_label['TOT'], c='r')
# ax2.set_yscale('log')
# ax2.set_ylim(ymin=70, ymax=1e4)
# ax2.annotate(r'E$_\mathrm{g}^{5}$', xy=(XX[3], bandgap_[3]+1e3), color='m')

# # Plotting sheet resistance
# R_300k = np.array(aln_algan_300k['R'], dtype=float) #/20000 #1/20000 is to rescale
# ax2.plot(XX, R_300k, 'o-', c='g')
# ax2.annotate(r'R', xy=(XX[3], R_300k[3]-1e3), color='g')

# Plotting LFOM
LFOM_300k = np.array(aln_algan_300k['LFOM'], dtype=float)/ref_LFOM 
ax.plot(XX, LFOM_300k, 'o-', c='r')
LFOM_300k = np.array(aln_algan_300k['LFOM_2DEG1e13'], dtype=float)/ref_LFOM 
ax.plot(XX, LFOM_300k, 'o-', c='b')
ax.set_ylabel(z_label['LFOMnorm_2'])
ax.axhline(y=1, c='k', ls='--')
#ax.annotate(r'LFOM$_{\mathrm{norm}}$', xy=(XX[3], LFOM_300k[3]-0.2), color='b')

plt2deg.save_figure(f'AlN50AlGaN_300K_mu_FOM_paper1.{FigFormat}', savefig=savefigure, show_plot=True,
                    fig=fig, CountFig=None, dpi=fig_save_dpi)